In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-28 23:07:38,952 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model')
model.init_sims(replace=True)

2017-06-28 23:07:39,531 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-28 23:07:45,796 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.* with mmap=None
2017-06-28 23:07:45,797 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.syn0.npy with mmap=None
2017-06-28 23:07:50,128 : INFO : setting ignored attribute syn0norm to None
2017-06-28 23:07:50,130 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.syn1neg.npy with mmap=None
2017-06-28 23:08:07,884 : INFO : setting ignored attribute cum_table to None
2017-06-28 23:08:07,895 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-28 23:08:11,425 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[(u'rs', 0.8790924549102783), (u'kk', 0.8604964017868042), (u'hahaha', 0.8581379652023315), (u'hahah', 0.8580098748207092), (u'hehe', 0.8567013740539551), (u'rsrs', 0.8455395698547363), (u'hehehe', 0.8401827812194824), (u'kkk', 0.8391278386116028), (u'hahahaha', 0.8145782947540283), (u'kkkk', 0.8133679032325745)]


In [59]:
print(model.most_similar(positive=[':D', ':/']))

[(u':(', 0.8271794319152832), (u';)', 0.8041931390762329), (u':)', 0.8021442890167236), (u':))', 0.7625139951705933), (u':c', 0.7616134881973267), (u'*-*', 0.7583409547805786), (u'=)', 0.7568512558937073), (u':3', 0.7542413473129272), (u':P', 0.7475375533103943), (u'^^', 0.744448184967041)]


In [60]:
print(model.most_similar(positive=['fofo', 'cute']))

[(u'fofinho', 0.7537943124771118), (u'lindinho', 0.7081056237220764), (u'meigo', 0.7032421827316284), (u'fofooo', 0.6875720620155334), (u'fofoooo', 0.6844723224639893), (u'fofo!', 0.6831097602844238), (u'cute!', 0.6810519695281982), (u'mordivel', 0.6774948835372925), (u'mord\xedvel', 0.6755068302154541), (u'fofo,', 0.6729605197906494)]


In [4]:
num_features= 300

In [5]:
df = pd.read_csv('~/personality-normalized-v2-word2vec.csv', encoding='utf-8')
df.shape

(1039, 186)

In [6]:
df = df[df.formatted_text.notnull()]
df.shape

(1028, 186)

In [7]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [8]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [9]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            if word in stopwords.words('portuguese'):
                continue
            tokens.append(word)
    return tokens

In [25]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [10]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [11]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-28 23:13:21,702 : WARNING : cannot compute similarity with no input [u'pokersport']


CPU times: user 3.31 s, sys: 59.6 ms, total: 3.37 s
Wall time: 3.73 s


In [12]:
del model

In [13]:
X_train_word_average.shape

(719, 300)

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    #cv=10,
    n_jobs=-1
)

## Extraversion evaluation

In [15]:
df.extraversion_m.value_counts()

0    520
1    508
Name: extraversion_m, dtype: int64

In [26]:
report(model_trainer.cv_results_)

Model with rank: 1
Mean validation score: 0.639 (std: 0.014)
Parameters: {'oob_score': True, 'bootstrap': True, 'n_estimators': 1000, 'random_state': 42, 'criterion': 'gini', 'max_features': 'log2', 'max_depth': None, 'class_weight': 'balanced'}



In [16]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    6.5s finished


CPU times: user 4.81 s, sys: 495 ms, total: 5.3 s
Wall time: 13.7 s


In [17]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [18]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,95,61
1,66,87


In [19]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.59      0.61      0.60       156
          1       0.59      0.57      0.58       153

avg / total       0.59      0.59      0.59       309



In [77]:
accuracy_score(yt, yp)

0.5889967637540453

In [78]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [20]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [21]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       364
          1       1.00      1.00      1.00       355

avg / total       1.00      1.00      1.00       719



In [22]:
print accuracy_score(ytt, ytp)

1.0


## Agreableness evaluation

In [82]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['agreeabeness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.8s finished


CPU times: user 4.82 s, sys: 115 ms, total: 4.94 s
Wall time: 32.6 s


In [83]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['agreeabeness_m']

In [84]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,56,85
1,57,111


In [85]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.50      0.40      0.44       141
          1       0.57      0.66      0.61       168

avg / total       0.53      0.54      0.53       309



In [86]:
accuracy_score(yt, yp)

0.54045307443365698

In [87]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [88]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['agreeabeness_m']

In [89]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       345
          1       1.00      1.00      1.00       374

avg / total       1.00      1.00      1.00       719



In [90]:
print accuracy_score(ytt, ytp)

1.0


## Conscientiousness evaluation

In [91]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['conscientiousness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   23.9s finished


CPU times: user 4.84 s, sys: 101 ms, total: 4.94 s
Wall time: 28.8 s


In [92]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['conscientiousness_m']

In [93]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,101,44
1,89,75


In [94]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.53      0.70      0.60       145
          1       0.63      0.46      0.53       164

avg / total       0.58      0.57      0.56       309



In [95]:
accuracy_score(yt, yp)

0.56957928802588997

In [96]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [97]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['conscientiousness_m']

In [98]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       373
          1       1.00      1.00      1.00       346

avg / total       1.00      1.00      1.00       719



In [99]:
print accuracy_score(ytt, ytp)

1.0


## Neuroticism evaluation

In [100]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['neuroticism_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.7s finished


CPU times: user 4.98 s, sys: 103 ms, total: 5.08 s
Wall time: 29.7 s


In [101]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['neuroticism_m']

In [102]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,101,51
1,90,67


In [103]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.53      0.66      0.59       152
          1       0.57      0.43      0.49       157

avg / total       0.55      0.54      0.54       309



In [104]:
accuracy_score(yt, yp)

0.5436893203883495

In [105]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [106]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['neuroticism_m']

In [107]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       378
          1       1.00      1.00      1.00       341

avg / total       1.00      1.00      1.00       719



In [108]:
print accuracy_score(ytt, ytp)

1.0


## Openess evaluation

In [109]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['openness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.5s finished


CPU times: user 4.86 s, sys: 105 ms, total: 4.96 s
Wall time: 31.5 s


In [110]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['openness_m']

In [111]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,82,59
1,67,101


In [112]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.55      0.58      0.57       141
          1       0.63      0.60      0.62       168

avg / total       0.59      0.59      0.59       309



In [113]:
accuracy_score(yt, yp)

0.59223300970873782

In [114]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [115]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['openness_m']

In [116]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       351
          1       1.00      1.00      1.00       368

avg / total       1.00      1.00      1.00       719



In [117]:
print accuracy_score(ytt, ytp)

1.0
